# EXERCISE 6
Number 4 – VGG16 Architecture with Muffin vs Chihuahua Dataset


In [3]:
# Lib imports
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
# DATASET DIRECTORY CONFIGURATION
train_dir = "train"  
test_dir = "test"    

In [8]:
# IMAGE PARAMETERS
IMG_SIZE = (224, 224)  
BATCH_SIZE = 32

In [11]:
# DATA PREPROCESSING & AUGMENTATION
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [12]:
# LOAD VGG16 BASE MODEL
base_model = VGG16(input_shape=(224,224,3), include_top=False, weights='imagenet')
base_model.trainable = False 

In [13]:
# ADD CUSTOM CLASSIFIER
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=outputs)


In [14]:
# COMPILE MODEL
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# TRAIN MODEL
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stop]
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 435s 4s/step - accuracy: 0.6705 - loss: 0.7973 - val_accuracy: 0.9228 - val_loss: 0.6502
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 378s 3s/step - accuracy: 0.8376 - loss: 0.6059 - val_accuracy: 0.9354 - val_loss: 0.5008
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 242s 2s/step - accuracy: 0.8968 - loss: 0.4827 - val_accuracy: 0.9439 - val_loss: 0.4150
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 302s 3s/step - accuracy: 0.9248 - loss: 0.3952 - val_accuracy: 0.9492 - val_loss: 0.3426
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.9369 - loss: 0.3420 - val_accuracy: 0.9492 - val_loss: 0.3010
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 410s 3s/step - accuracy: 0.9398 - loss: 0.3010 - val_accuracy: 0.9545 - val_loss: 0.2747
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 241s 2s/step - accuracy: 0.9435 - loss: 0.2775 - val_accuracy: 0.9566 - val_loss: 0.2522
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 235s 2s/step - accuracy: 0.9509 - loss: 0.2546 - val_accu

In [16]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9603 - loss: 0.2011
Test Accuracy: 0.9603


In [17]:
# SAVE MODEL
model.save("exercise_6_vgg16model_Esponilla-Valdez.h5")

In [18]:
# INFERENCE FUNCTION
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path="exercise_6_vgg16model_Esponilla-Valdez.h5"):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    if pred >= 0.5:
        label = "Muffin"
        confidence = pred
    else:
        label = "Chihuahua"
        confidence = 1.0 - pred
    print(f"Prediction: {label} (confidence: {confidence:.2f})")

In [19]:
# RUN 1 AND RUN 2 EXAMPLES
predict_image("test/chihuahua/img_0_18.jpg")
predict_image("test/muffin/img_0_10.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step


Prediction: Chihuahua (confidence: 0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
Prediction: Muffin (confidence: 1.00)
